In [1]:
import sys
sys.path.append("../")

from src.modelling.feature_extraction import Feature_Extraction as FE
from src.modelling.data_validation import Data_Validation as DV
from src.modelling.model_error_analysis import ModelErrorAnalysis as MEA

from src.data.etl_layer import ETL
from src.data.eda_layer import EDA

2024-03-18 13:08:23.810006: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 13:08:23.810038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 13:08:23.810978: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-18 13:08:23.816692: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-18 13:08:24.576804: W tensorflow/compiler/tf2

# Data preprocessing

In [2]:
# Raw dataset load and processing

sales_etl = ETL("../data/raw/sales_train.csv")

sales_etl.transform(["item_cnt_day"], ["item_cnt_day"])
sales_df = sales_etl.get_data()


2935849 rows and 6 columns has been read from sales_train.csv

306477 outliers were found for the item_cnt_day column.

306477 outliers in column item_cnt_day processed.

Number of rows with missing values: 0

0 rows with N/A values processed.

Number of conflicting or duplicated rows: 6

6 inconsistent or duplicated rows processed.

Сolumns with presumably the wrong data type: 
not numeric type columns: [],
not date type columns: ['date'],
not string type columns: []

DataFrame column types processed.


In [3]:
# loading the full processed dataset motnhly_sales and type conversion for 'month' column
monthly_df_etl = ETL(data_path="../data/processed/monthly_sales_full_processed.csv")
monthly_df_etl.data_type_conversion()
monthly_df = monthly_df_etl.get_data()


1609124 rows and 4 columns has been read from monthly_sales_full_processed.csv

Сolumns with presumably the wrong data type: 
not numeric type columns: [],
not date type columns: ['month'],
not string type columns: []

DataFrame column types processed.


## Feature Extraction

In [8]:
train_df_fe = FE(sales_df, monthly_df)
train_df = train_df_fe.get_fe_df()
train_df.head()

,shop_id,item_id,item_cnt_month,mean_month_price,month_num
0,0,32,1.870712,221.0,1
1,0,33,3.000000,347.0,1
2,0,35,1.000000,247.0,1
3,0,43,1.000000,221.0,1
4,0,51,2.000000,128.5,1


In [ ]:
# Feature